# lecture2 データの集計

In [3]:
import numpy as np
import pandas as pd

## Q1

入力リストの全ての要素の平均    
入力リストの全ての要素の中央値  
入力リストの全ての要素の分散  

を一つのリストに格納して出力

In [6]:
def avg_med_var(input_list):
    mean_value=0 #平均値を初期化
    med_value=0 # 中央値を初期化
    var_value=0 # 分散を初期化
    results=[] # 結果のリスト
        
    input_list.sort() # リストの要素を昇順に並び替え
    mid_idx = len(input_list)//2 # リストの中央（長さの半分）のインデックス（必要であれば利用する）
    if len(input_list)%2==0:
        med_value= (input_list[mid_idx-1] + input_list[mid_idx])/2
    else:
        med_value = input_list[mid_idx]
    
    mean_value = sum(input_list)/len(input_list) #平均値
    mse_value = 0 # データの値と平均値の差の2乗和を初期化
    for i in range(len(input_list)):
        mse_value += (input_list[i] - mean_value)**2
    var_value = mse_value / len(input_list)
    
    results.append(mean_value) 
    results.append(med_value) 
    results.append(var_value)
    return results   

In [7]:
# avg_med_var関数の結果を検証するための関数
import numpy as np
def avg_med_var_test(input_list):
    return [np.mean(input_list), np.median(input_list), np.var(input_list)]

In [8]:
# 偶数の場合の検算
assert avg_med_var([1,4,3,2,5,6,8,7,9,10]) == avg_med_var_test([1,4,3,2,5,6,8,7,9,10])

In [9]:
# 奇数の場合の検算
assert avg_med_var([1,4,3,2,5,6,8,7,9,10,11]) == avg_med_var_test([1,4,3,2,5,6,8,7,9,10,11])

ここで答えが違う場合は"Assersion error"が出る。

## Q2

In [12]:
import pandas as pd
score = pd.read_csv('ex2/user_score.csv')
score.head(5)

,user,kokugo,shakai,sugaku,rika
0,1,30,43,51,63
1,2,39,21,49,56
2,3,29,30,23,57
3,4,95,87,77,100
4,5,70,71,78,67


In [13]:
score.describe()

,user,kokugo,shakai,sugaku,rika
count,166.000000,166.000000,166.000000,166.000000,166.000000
mean,83.500000,52.331325,39.602410,45.614458,49.885542
std,48.064193,21.835858,21.456335,24.236127,21.408130
min,1.000000,0.000000,4.000000,0.000000,9.000000
25%,42.250000,35.000000,23.000000,28.000000,31.000000
50%,83.500000,53.000000,35.000000,43.000000,49.000000
75%,124.750000,70.000000,53.750000,63.750000,65.750000
max,166.000000,96.000000,90.000000,100.000000,100.000000


### Q2-1

データフレームscoreから各教科の得点がすべて平均点以上であるようなユーザの数を返すabove_mean()関数の作成。

In [14]:
def above_mean(df):
    return len(df[(df.kokugo>df.kokugo.mean())&(df.shakai>df.shakai.mean())&(df.sugaku>df.sugaku.mean())&(df.rika>df.rika.mean())])

In [15]:
above_mean(score)

46

## Q2-2

ユーザごとの全教科の得点の総和の値を表すsum列をデータフレームscoreに新たに追加したデータフレームを返すscore_sum()関数の作成。

In [17]:
def score_sum(df):
    df['sum']= df[['kokugo','shakai','sugaku','rika']].sum(axis=1)
    return df

In [18]:
score = score_sum(score)
score.head(5)

,user,kokugo,shakai,sugaku,rika,sum
0,1,30,43,51,63,187
1,2,39,21,49,56,165
2,3,29,30,23,57,139
3,4,95,87,77,100,359
4,5,70,71,78,67,286


全教科の合計得点の上位3人の'user'を1位、2位、3位の順に要素として持つリストを返すscore_top3()関数の作成。

In [20]:
def score_top3(df):
    sorted_df = df.sort_values(by='sum', ascending=False)
    results=[]
    results.append(sorted_df.user.values[0])
    results.append(sorted_df.user.values[1])
    results.append(sorted_df.user.values[2])
    return results

In [21]:
score_top3(score)

[39, 4, 50]

## Q2-3

データフレームscoreのsum列を対象に、階級幅50として各階級の度数を要素として持つリストを返すscore_hist()関数の作成。

In [23]:
def score_hist(df):
    results=[]
    results.append(df.loc[(df['sum']>=0) & (df['sum']<=50), 'sum'].count())
    results.append(df.loc[(df['sum']>=51)&(df['sum']<=100), 'sum'].count())
    results.append(df.loc[(df['sum']>=101) & (df['sum']<=150), 'sum'].count())
    results.append(df.loc[(df['sum']>=151) & (df['sum']<=200), 'sum'].count())
    results.append(df.loc[(df['sum']>=201) & (df['sum']<=250), 'sum'].count())
    results.append(df.loc[(df['sum']>=251) & (df['sum']<=300), 'sum'].count())
    results.append(df.loc[(df['sum']>=301) & (df['sum']<=350), 'sum'].count())
    results.append(df.loc[(df['sum']>=351) & (df['sum']<=400), 'sum'].count())
    return results
score_hist(score)

[4, 20, 39, 35, 29, 24, 13, 2]

## Q2-4

In [26]:
user_class = pd.read_csv('ex2/user_class.csv')
user_class.head(5)

,user,class
0,1,C
1,2,C
2,3,B
3,4,C
4,5,A


'user'の列の値をキーに、2つのデータフレーム、user_classとscore、を結合して新しいデータフレームを作成した上で、データフレームの`groupby`メソッドを使って、'class'をインデックス、'class'ごとの'sum'の最高値を列として持つシリーズオブジェクトを返す`score_by_class()`関数を作成。

In [27]:
def score_by_class(df1, df2):
    df3 =pd.merge(df1, df2, on='user')
    return df3[['class', 'sum']].groupby(['class'], as_index=True).max()['sum']

In [28]:
score_by_class(score, user_class).to_dict()

{'A': 361, 'B': 349, 'C': 359}